#Dia 5 - Desenvolvendo API

In [3]:
# Importei as bibliotecas necessárias. 
# Flask cria a conexão e disponibiliza online
# pickle carrega o modelo de machine learning

import numpy as np
from flask import Flask
import pickle
import pandas as pd
import requests
import json

In [5]:
# carregando o modelo de machine learning

modelo = pickle.load(open('modelo_ptr.pk1','rb'))

In [11]:
# Perguntando para quem será recomendado

pos = int(input('Para quem recomendar: '))

In [12]:
# Criando a aplicação
app = Flask(__name__)

# endpoint mostrando que o site está funcionando
@app.route('/')
def mensagem():
    return "FUNCIONANDO"

# endpoint fazendo a indicação
@app.route('/indica')
def indica():
    # dados salvo na nuvem
    df = requests.get('https://recomendacao-95a6d-default-rtdb.firebaseio.com/.json')
    df = df.json()
    df = pd.DataFrame(df)
    recomendacao = df.pivot_table(index='user_id', columns='movie_title', values='rating').fillna(0)
    distancia, vizinhos = modelo.kneighbors(recomendacao.iloc[pos].values.reshape(1,-1), n_neighbors=6)
    ds_usuario = recomendacao.iloc[pos].to_frame()
    vizinhanca = ds_usuario
    for i in range(1, 6):
        vizinho_prox = recomendacao.index[vizinhos.flatten()[i]]
        ds_vizinho = recomendacao.loc[vizinho_prox].to_frame()
        vizinhanca = pd.merge(vizinhanca, ds_vizinho[vizinho_prox], on='movie_title')
    pos1 = recomendacao.index[vizinhos.flatten()[1]]
    pos2 = recomendacao.index[vizinhos.flatten()[2]]
    pos3 = recomendacao.index[vizinhos.flatten()[3]]
    pos4 = recomendacao.index[vizinhos.flatten()[4]]
    pos5 = recomendacao.index[vizinhos.flatten()[5]]
    vizinhanca_df = vizinhanca.reset_index()
    vizinhanca_df.rename(columns={pos + 1 : 'coluna'}, inplace=True)
    vizinhanca_df['SOMA'] = vizinhanca_df[pos1] + vizinhanca_df[pos2] + vizinhanca_df[pos3] + vizinhanca_df[pos4] + vizinhanca_df[pos5]
    indicacao = vizinhanca_df.sort_values(by='SOMA', ascending=False)
    indicacao.query('coluna == 0')
    resposta = str(list(indicacao['movie_title'][:5]))
    return resposta

if __name__ == "__main__":
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [31/May/2024 19:40:14] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [31/May/2024 19:40:14] "GET /favicon.ico HTTP/1.1" 404 -
127.0.0.1 - - [31/May/2024 19:40:27] "GET / HTTP/1.1" 200 -
[2024-05-31 19:40:39,097] ERROR in app: Exception on /indica [GET]
Traceback (most recent call last):
  File "c:\Users\patri\AppData\Local\Programs\Python\Python311\Lib\site-packages\flask\app.py", line 2190, in wsgi_app
    response = self.full_dispatch_request()
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\patri\AppData\Local\Programs\Python\Python311\Lib\site-packages\flask\app.py", line 1486, in full_dispatch_request
    rv = self.handle_user_exception(e)
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\patri\AppData\Local\Programs\Python\Python311\Lib\site-packages\flask\app.py", line 1484, in full_dispatch_request
    rv = self.dispatch_request()
         ^^^^^^^^^^^^^^^^^^^^^^^
  File "c:\Users\patri\AppData\Loca